
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Advanced Delta Lake Features

Now that you feel comfortable performing basic data tasks with Delta Lake, we can discuss a few advanced features unique to Delta Lake. We are going to talk about Liquid Clustering, Optimization, and Versioning in Delta Lake.

Note that while some of the keywords used here aren't part of standard ANSI SQL, all Delta Lake operations can be run on Databricks using SQL

## Learning Objectives
By the end of this lesson, you should be able to:
* Use **`CLUSTER BY`** for liquid clustering
* Use **`OPTIMIZE`** to manually trigger liquid clustering
* Review a history of table transactions
* Query and roll back to previous table version
* Describe how to enable **`Predictive Optimization`**

**Resources**
* <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-optimize.html" target="_blank">Delta Optimize - Databricks Docs</a>
* <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-vacuum.html" target="_blank">Delta Vacuum - Databricks Docs</a>

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-7

## Liquid Clustering
Delta Lake liquid clustering replaces table partitioning and ZORDER to simplify data layout decisions and optimize query performance. Liquid clustering provides flexibility to redefine clustering keys without rewriting existing data, allowing data layout to evolve alongside analytic needs over time.

Databricks recommends using liquid clustering for all new Delta tables.

We enable liquid clustering on a table by using **`CLUSTER BY`**.

Run **`DESCRIBE events`** and note the names of the columns.

In [0]:
DESCRIBE events;

There are [many reasons](https://docs.databricks.com/en/delta/clustering.html#what-is-liquid-clustering-used-for) to use liquid clustering on a table. We know the **`events`** table will be growing quickly and will require maintenance and tuning, so we are going to enable liquid clustering for this table. Now, we could have enable liquid clustering at the time the table was created by adding **`CLUSTER BY`** to the **`CREATE TABLE`** statement, like this:

In [0]:
CREATE OR REPLACE TABLE events_liquid 
CLUSTER BY (user_id) AS 
SELECT * 
FROM events;

However, we can also add liquid clustering to an existing table using **`ALTER TABLE`**.

In [0]:
ALTER TABLE events
CLUSTER BY (user_id);

When we run **`DESCRIBE events`**, we see the column(s) on which we are currently clustering under **`Clustering Information`**.

In [0]:
DESCRIBE events;

## Choosing Clustering Keys
Databricks recommends choosing clustering keys based on commonly used query filters. Clustering keys can be defined in any order. 

In the **`CLUSTER BY`** above, we chose **`user_id`** as the clustering key, but we may also want to add **`device`**. Note that we can change clustering keys, as needed, by altering the table in the future.

With liquid clustering, we no longer have to worry about how we have data partitioned or deal with the complexities of using zorder. We get the benefits of both without the struggle.

## Triggering Liquid Clustering
Liquid clustering is incremental, meaning that data is only rewritten as necessary to accommodate data that needs to be clustered. Data files with clustering keys that do not match data to be clustered are not rewritten.

For best performance, Databricks recommends scheduling regular **`OPTIMIZE`** jobs to cluster data. For tables experiencing many updates or inserts, Databricks recommends scheduling an **`OPTIMIZE`** job every one or two hours. Because liquid clustering is incremental, most **`OPTIMIZE`** jobs for clustered tables run quickly.

In [0]:
OPTIMIZE events;

## Creating a Delta Table with History

In the next cell, we create a table and run a handful of commands that make updates to the table. As you're waiting for this query to run, see if you can identify the total number of transactions being executed.

In [0]:
CREATE TABLE students 
  (id INT, name STRING, value DOUBLE);
  
INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

INSERT INTO students
VALUES 
  (4, "Ted", 4.7),
  (5, "Tiffany", 5.5),
  (6, "Vini", 6.3);
  
UPDATE students 
SET value = value + 1
WHERE name LIKE "T%";

DELETE FROM students 
WHERE value > 6;

CREATE OR REPLACE TEMP VIEW updates(id, name, value, type) AS VALUES
  (2, "Omar", 15.2, "update"),
  (3, "", null, "delete"),
  (7, "Blue", 7.7, "insert"),
  (11, "Diya", 8.8, "update");
  
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *;

## The Delta Log
Each change to a table results in a new entry being written to the Delta Lake transaction log. 

The command, `DESCRIBE HISTORY` allows us to see this log

In [0]:
DESCRIBE HISTORY students;

## Deletion Vectors
Note that the log includes an **OPTIMIZE** operation, yet we never called **`OPTIMIZE`** on the **`students`** table. If you open the `operationParameters` for the **`OPTIMIZE`** operation, you will see that `auto: true`. This is because Deletion Vectors triggered auto-compaction. When we delete rows from a table, Deletion Vectors mark those rows for deletion but do not re-write the underlying Parquet files. This helps reduce the so-called small file problem, where a table is made up of a large number of small Parquet files. However, Deletion Vectors will trigger auto-compaction, and the underlying files are re-written.



## Delta Lake Time Travel

Delta Lake gives us the opportunity to query tables at any point in the transaction log. These time travel queries can be performed by specifying either the version number or the timestamp.

**NOTE**: In most cases, you'll use a timestamp to recreate data at a time of interest. For our demo we'll use version.

In [0]:
SELECT * 
FROM students VERSION AS OF 3;



What's important to note about time travel is that we're not recreating a previous state of the table by undoing transactions against our current version; rather, we're just querying all those data files that were indicated as valid as of the specified version.

## Rollback Versions

Suppose you're typing up a query to manually delete some records from a table and you accidentally delete all records.

In [0]:
DELETE FROM students;

From the output above, we can see that 4 rows were removed.

Let's confirm this below.

In [0]:
SELECT * 
FROM students;

Deleting all the records in your table is probably not a desired outcome. Luckily, we can simply rollback this commit.

In [0]:
RESTORE TABLE students TO VERSION AS OF 8;

In [0]:
-- Confirm table has been 'Restored'
SELECT * 
FROM students;

Note that a **`RESTORE`** <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-restore.html" target="_blank">command</a> is recorded as a transaction; you won't be able to completely hide the fact that you accidentally deleted all the records in the table, but you will be able to undo the operation and bring your table back to a desired state.

## Predictive Optimization
Predictive Optimization is a feature that can be enabled that takes away the necessity for manually performing **`OPTIMIZE`** and **`VACUUM`**.

With predictive optimization enabled, Databricks automatically identifies tables that would benefit from maintenance operations and runs them for the user. Maintenance operations are only run as necessary, eliminating both unnecessary runs for maintenance operations and the burden associated with tracking and troubleshooting performance.

Predictive Optimization can be enabled at the account level, or on catalogs and schemas, as long as the user enabling this feature has the correct permission level. The feature is inherited by all lower-level objects, but it can be enabled/disabled on those objects, as needed.

To check whether Predictive Optimization is enabled on a table, run **`DESCRIBE EXTENDED <table name>`**:



In [0]:
DESCRIBE EXTENDED events;


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>